# Real-Time Object Detection App

This notebook demonstrates a real-time object detection application using:
- YOLOv8 for object detection
- OpenCV for image processing
- Streamlit for the web interface

The app will be able to:
1. Detect objects from webcam feed
2. Process uploaded videos
3. Display detection results in real-time

We'll develop the core functionality here and then integrate it into a Streamlit app.

In [ ]:
# Install required packages using uv
!uv pip install ultralytics opencv-python streamlit ipykernel

# Note: If uv is not installed, you can install it with:
# pip install uv

In [ ]:
# Import necessary libraries
import cv2
import numpy as np
from ultralytics import YOLO
import time
import os

# For visualization
import matplotlib.pyplot as plt
from PIL import Image

: 

In [ ]:
# Load the YOLOv8 model
# Check if the model file exists in the current directory
model_path = "yolov8s.pt"
if not os.path.exists(model_path):
    print(f"Model file {model_path} not found. Downloading...")
    model = YOLO("yolov8s")  # This will download the model if not present
else:
    print(f"Loading model from {model_path}")
    model = YOLO(model_path)

# Print model information
print(f"Model loaded: {model}")

: 

In [ ]:
# Function to perform object detection on an image
def detect_objects(image, model, conf=0.25):
    """
    Perform object detection on an image using the YOLOv8 model.
    
    Args:
        image: Input image (numpy array)
        model: YOLOv8 model
        conf: Confidence threshold (default: 0.25)
        
    Returns:
        processed_img: Image with detection boxes
        results: Detection results
    """
    # Perform inference
    results = model(image, conf=conf)
    
    # Process results
    processed_img = results[0].plot()
    
    return processed_img, results[0]

# Function to display detection results
def display_results(results):
    """
    Display detection results.
    
    Args:
        results: Detection results from YOLOv8
    """
    # Get detected objects
    boxes = results.boxes.cpu().numpy()
    
    if len(boxes) == 0:
        print("No objects detected")
        return
    
    # Display detection information
    print(f"Detected {len(boxes)} objects:")
    
    # Get class names
    class_names = results.names
    
    for i, box in enumerate(boxes):
        cls_id = int(box.cls[0])
        conf = box.conf[0]
        label = class_names[cls_id]
        
        print(f"{i+1}. {label}: {conf:.2f}")

: 

In [ ]:
# Function to process video stream (webcam or video file)
def process_video(source=0, model=None, conf=0.25, display=True):
    """
    Process video stream from webcam or video file and perform object detection.
    
    Args:
        source: Video source (0 for webcam, or path to video file)
        model: YOLOv8 model
        conf: Confidence threshold
        display: Whether to display the processed video
        
    Returns:
        None
    """
    if model is None:
        print("Model not loaded. Please load a model first.")
        return
    
    # Open video capture
    cap = cv2.VideoCapture(source)
    
    if not cap.isOpened():
        print(f"Error: Could not open video source {source}")
        return
    
    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    print(f"Video properties: {width}x{height} at {fps} FPS")
    
    # Process video frames
    frame_count = 0
    start_time = time.time()
    
    try:
        while True:
            # Read frame
            ret, frame = cap.read()
            
            if not ret:
                print("End of video stream")
                break
            
            # Perform object detection
            processed_frame, results = detect_objects(frame, model, conf)
            
            # Calculate FPS
            frame_count += 1
            elapsed_time = time.time() - start_time
            current_fps = frame_count / elapsed_time
            
            # Add FPS information to the frame
            cv2.putText(processed_frame, f"FPS: {current_fps:.2f}", (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
            if display:
                # Display the processed frame
                cv2.imshow("Object Detection", processed_frame)
                
                # Break loop on 'q' key press
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
    
    finally:
        # Release resources
        cap.release()
        if display:
            cv2.destroyAllWindows()
        
        print(f"Processed {frame_count} frames in {elapsed_time:.2f} seconds")
        print(f"Average FPS: {frame_count / elapsed_time:.2f}")